In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install emoji unidecode

**preprocess data **

In [0]:
import nltk
nltk.download('punkt')
from nltk.tokenize import MWETokenizer, word_tokenize, RegexpTokenizer
import re
import nltk
import unicodedata

multiple_punctuation_pattern = re.compile(r"([\"\.\?\!\,\:\;\-])(?:[\"\.\?\!\,\:\;\-]){1,}")
word_tokenizer = MWETokenizer(separator='')
multiple_emoji_pattern = re.compile(u"(["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u00a9"
        u"\u00ae"
        u"\u2000-\u3300"
        "]){1,}", flags= re.UNICODE )

normalizer = {'òa': 'oà',
              'óa': 'oá',
              'ỏa': 'oả',
              'õa': 'oã',
              'ọa': 'oạ',
              'òe': 'oè',
              'óe': 'oé',
              'ỏe': 'oẻ',
              'õe': 'oẽ',
              'ọe': 'oẹ',
              'ùy': 'uỳ',
              'úy': 'uý',
              'ủy': 'uỷ',
              'ũy': 'uỹ',
              'ụy': 'uỵ',
              'Ủy': 'Uỷ'}
correct_mapping = {
      "m": "mình",
      "mik": "mình",
      "ko": "không",
      "k": " không ",
      "kh": "không",
      "khong": "không",
      "kg": "không",
      "khg": "không",
      "tl": "trả lời",
      "r": "rồi",
      "ok": "tốt",
      "dc": "được",
      "vs": "với",
      "đt": "điện thoại",
      "thjk": "thích",
      "thik": "thích",
      "qá": "quá",
      "trể": "trễ",
      "bgjo": "bao giờ",
      "''": '"',
      "``": '"'
}

def normalize_text(text):
  for absurd, normal in normalizer.items():
    text = text.replace(absurd, normal)

  # for l in vn_location:
  #   text = text.replace(l, ' location ')

  return text

def tokmap(tok):
  if tok.lower() in correct_mapping:
      return correct_mapping[tok.lower()]
  else:
      return tok

def preprocess(text):
  global i
  text = multiple_emoji_pattern.sub(r"\g<1> ", text) # \g<1>
  text = multiple_punctuation_pattern.sub(r" \g<1> ", text)
  text = unicodedata.normalize("NFC", text)
  text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b(\/)?', 'url', text)
  text = re.sub("\.", " . ", text)
  text = re.sub("'", "' ", text)
  text = re.sub('"', '" ', text)
  text = re.sub('/', ' / ', text)
  text = re.sub('-', ' - ', text)
  text = re.sub(',', ' , ', text)
  text = re.sub(r'\s{2,}', ' ', text)
  text = normalize_text(text)
  # text = re.sub(r'\#[^\s]+', ' hastag ', text)
  text = re.sub(r'(|\s)([\d]+k)(\s|$)', ' cureency_k ', text)
  text = re.sub(r'(([\d]{2,4}\s){2,}([\d]+)?|(09|01|[2|6|8|9]|03)+([0-9]{8})\b)', ' phone_number ', text)
  # text = re.sub(r'\d', "_digit", text)
  tokens = word_tokenizer.tokenize(word_tokenize(text))
  tokens = list(map(tokmap, tokens))
  # return tokens
  return ' '.join(tokens)

print(preprocess('Địa chỉ : 103 d9 ngõ 63 , thái thịnh, đống đa, hà nội.'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Địa chỉ : _digit_digit_digit d_digit ngõ _digit_digit , thái thịnh , đống đa , hà nội .


**Feature extraction model**

In [0]:
import re
import string
import unidecode
import numpy as np
import scipy.sparse as sp
import emoji
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import SelectFromModel


class RemoveConsecutiveSpaces(BaseEstimator, TransformerMixin):
    # def remove_consecutive_spaces(self, s):
    #     return ' '.join(s.split())

    def transform(self, x):
        return [re.sub(r'\s+', ' ', s) for s in x] # self.remove_consecutive_spaces(s)

    def fit(self, x, y=None):
        return self


class RemovePunct(BaseEstimator, TransformerMixin):
    non_special_chars = re.compile('[^A-Za-z0-9 ]+')

    def remove_punct(self, s):
        return re.sub(self.non_special_chars, '', s)

    def transform(self, x):
        return [self.remove_punct(s) for s in x]

    def fit(self, x, y=None):
        return self



class RemoveTone(BaseEstimator, TransformerMixin):
    def remove_tone(self, s):
        return unidecode.unidecode(s)

    def transform(self, x):
        return [self.remove_tone(s) for s in x]

    def fit(self, x, y=None):
        return self


class NumWordsCharsFeature(BaseEstimator, TransformerMixin):
    def count_char(self, s):
        return len(s)

    def count_word(self, s):
        return len(s.split())

    def transform(self, x):
        count_chars = sp.csr_matrix([self.count_char(s) for s in x], dtype=np.float64).transpose()
        count_words = sp.csr_matrix([self.count_word(s) for s in x], dtype=np.float64).transpose()

        return sp.hstack([count_chars, count_words])

    def fit(self, x, y=None):
        return self


class ExclamationMarkFeature(BaseEstimator, TransformerMixin):
    def count_exclamation(self, s):
        count = s.count('!') + s.count('?')
        return count / (1 + len(s.split()))

    def transform(self, x):
        counts = [self.count_exclamation(s) for s in x]
        return sp.csr_matrix(counts, dtype=np.float64).transpose()

    def fit(self, x, y=None):
        return self


class NumCapitalLettersFeature(BaseEstimator, TransformerMixin):
    def count_upper(self, s):
        n_uppers = sum(1 for c in s if c.isupper())
        return n_uppers / (1 + len(s))

    def transform(self, x):
        counts = [self.count_upper(s) for s in x]
        return sp.csr_matrix(counts, dtype=np.float64).transpose()

    def fit(self, x, y=None):
        return self


class NumLowercaseLettersFeature(BaseEstimator, TransformerMixin):
    def count_lower(self, s):
        n_lowers = sum(1 for c in s if c.islower())
        return n_lowers / (1 + len(s))

    def transform(self, x):
        counts = [self.count_lower(s) for s in x]
        return sp.csr_matrix(counts, dtype=np.float64).transpose()

    def fit(self, x, y=None):
        return self


class NumPunctsFeature(BaseEstimator, TransformerMixin):
    def count(self, l1, l2):
        return sum([1 for x in l1 if x in l2])

    def count_punct(self, s):
        n_puncts = self.count(s, set(string.punctuation))
        return n_puncts / (1 + len(s))

    def transform(self, x):
        counts = [self.count_punct(s) for s in x]
        return sp.csr_matrix(counts, dtype=np.float64).transpose()

    def fit(self, x, y=None):
        return self

class NumHashTagFeature(BaseEstimator, TransformerMixin):
  def count(self, s):
    multiple_star = re.compile(r"[\#]+")
    n_hash = len(multiple_star.findall(s))
    return n_hash / (1 + len(s.split() ))

  def transform(self, x):
    multiple_star = re.compile(r"[\#]+")
    counts = [ self.count(s) for s in x]
    return sp.csr_matrix(counts, dtype=np.float64).transpose()
  
  def fit(self, x, y=None):
    return self


class NumEmojiFeature(BaseEstimator, TransformerMixin):
    def count_emoji(self, s):
        emoji_list = []
        for c in s:
            if c in emoji.UNICODE_EMOJI:
                emoji_list.append(c)
        return len(emoji_list) / (1 + len(s.split()))

    def transform(self, x):
        counts = [self.count_emoji(s) for s in x]
        return sp.csr_matrix(counts, dtype=np.float64).transpose()

    def fit(self, x, y=None):
        return self

feature_extraction = Pipeline([
    ('remove_spaces', RemoveConsecutiveSpaces()),
    ('features', FeatureUnion([
        ('custom_features_pipeline', Pipeline([
            ('custom_features', FeatureUnion([
                ('f01', NumWordsCharsFeature()),
                ('f02', NumCapitalLettersFeature()),
                ('f03', ExclamationMarkFeature()),
                ('f04', NumPunctsFeature()),
                ('f05', NumLowercaseLettersFeature()),
                ('f06', NumEmojiFeature()),
            ], n_jobs=-1)),
            ('scaler', StandardScaler(with_mean=False))
        ])),
        ('word_char_features_pipeline', Pipeline([
            ('word_char_features', FeatureUnion([
                ('with_tone', Pipeline([
                    ('tf_idf_word', TfidfVectorizer(ngram_range=(1, 3), norm='l2', min_df=2)) # lowercase=False
                ])),
                ('with_tone_char', TfidfVectorizer(ngram_range=(1, 6), norm='l2', min_df=2, analyzer='char')),
                ('with_tone_char_wb', TfidfVectorizer(ngram_range=(1, 6), norm='l2', min_df=2, analyzer='char_wb')),
                ('without_tone', Pipeline([
                    ('remove_tone', RemoveTone()),
                    ('without_tone_features', FeatureUnion([
                        ('tf_idf', Pipeline([
                            ('word', TfidfVectorizer(ngram_range=(1, 3), norm='l2', min_df=2) )
                        ])),
                        ('tf_idf_char', TfidfVectorizer(ngram_range=(1, 6), norm='l2', min_df=2, analyzer='char')),
                        ('tf_idf_char_wb',
                          TfidfVectorizer(ngram_range=(1, 6), norm='l2', min_df=2, analyzer='char_wb'))
                    ], n_jobs=-1))
                ]))
            ], n_jobs=-1))
        ]))
    ], n_jobs=-1)),
])


**LOAD train data**

In [0]:
from sklearn import utils as skutils


def maping_lable(lable):
  if lable == '__label__post_khong_ban':
    return 0
  return 1


def load_train_test():
  with open('./sell_detection_train.v1.0.txt') as f:
    lines = f.read().strip().split('\n')
    x_train = [t.split(' ', 1)[1] for t in lines]
    x_train = [ preprocess(descriptions) for descriptions in x_train]

    y_train = [t.split(' ', 1)[0] for t in lines]
    y_train = list(map(maping_lable, y_train))

  # with open('10kdigit_encode.txt') as f:
  #   lines = f.read().strip().split('\n')
  #   x_test = [t.split(' ', 1)[1] for t in lines]
  #   y_test = [t.split(' ', 1)[0] for t in lines]
  #   y_test = list(map(maping_lable, y_test))
  # x_train.extend(x_test)
  # y_train.extend(y_test)
  return skutils.Bunch(x_train=x_train, y_train=y_train) #, x_test=x_test, y_test=y_test)

data = load_train_test()

print(len(data.y_train), len(data.x_train) )

60000 60000


**Load test data**

In [0]:
with open('sell_detection_test.v1.0.txt') as f:
    sell_detection_test = f.read().strip().split('\n')
    sell_detection_test = [ preprocess(e) for e in sell_detection_test]
print(sell_detection_test[:10])
print(len(sell_detection_test))


['\ufeffBéo mà .', 'Minh la tai nguyen', 'Moi bat được con nay nhau được Hok', 'Bữa ăn sáng trên bầu trời', '❤ ️ 01 / 01 / 2013 - 01 / 01 / 2018 : Gặp được nhau đã là cái duyên , đến với nhau , bên cạnh nhau , quan tâm nhau . có lẽ là do số phận ! 5 năm thanh xuân có nhau . lãi được 1 cục 💎 P / s : Mỗi năm lãi đều như vậy thì lời to rồi 😍 # 5year_anniversary # HTloveMN # kếhoạch5nămlần1', 'Sơri Võ', 'ngu thoj chuc ka nha ngu ngon', 'Thực hiện chính sách đại lý mới nhằm chống bán phá giá để bảo vệ thương hiệu và các đại lý của Istitch . Kể từ nay Istitch chỉ bán cho 1 đại lý ở mỗi tỉnh thành . Các quý đại lý có nhu cầu xin vui lòng liên hệ email : thanhhongdo @ gmail . com để lấy báo giá riêng nhé ! Xin cám ơn .', 'Snowntown Sài Gòn ❤ ️⛄ ️❄ ️', 'ດົນໆໄປເທື່ອ . ມ່ວນຫຼາຍມື້ນີ້']
98893


In [0]:
x_train_feature = feature_extraction.fit_transform(data.x_train)
print(x_train_feature.shape)
# x_test_feature = feature_extraction.transform(data.x_test)
# print(x_test_feature.shape)

(60000, 2134505)


In [0]:
x_train_feature = feature_extraction.transform(data.x_train)
print(x_train_feature.shape)

(60000, 2134505)


In [0]:
# import pickle
# with open ('feature_extraction.pickle', 'wb') as f:
#   pickle.dump(feature_extraction, f)

In [0]:
# import pickle

# with open ('feature_extraction.pickle', 'rb') as f:
#   feature_extraction = pickle.load(f) 

In [0]:
x_test_feature = feature_extraction.transform(sell_detection_test)
print(x_test_feature.shape)

(98893, 2134505)


In [0]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

linear_svm = LinearSVC()
linear_svm.fit(x_train_feature, data.y_train)

selection_model = SelectFromModel(linear_svm, prefit=True)

X_train_selection = selection_model.transform(x_train_feature)
X_test_selection = selection_model.transform(x_test_feature)
print(X_train_selection.shape)
print(X_test_selection.shape)


LinearSVM = LinearSVC()
LinearSVM.fit(X_train_selection, data.y_train)

predicted = LinearSVM.predict(X_test_selection)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


(60000, 560858)
(98893, 560858)


In [0]:
def reverse_maping_lable(lable):
  if lable == 1:
    return '__label__post_ban_hang'
  return '__label__post_khong_ban'

predicted_origin_lables = list(map(reverse_maping_lable, predicted))
with open('sell_detection_team03_solution1.result.txt', 'w') as f:
  f.write('\n'.join(predicted_origin_lables))